# **RAG의 한계를 극복하기 위한 매우 효과적이고 현대적인 접근 방식인 중 하나가 ReAct(Reason + Act) 프레임워크 기반의 에이전트(Agent)입니다. **

Function Calling (또는 LangChain의 Tool 사용)은 이 에이전트가 "행동(Act)"하는 핵심 방법 중 하나입니다.

**이 방법이 왜 좋은가?**

- 조건부 실행: 기존 RAG는 질문이 들어오면 무조건 검색(Retrieve)하고 생성(Generate)합니다. 하지만 ReAct 에이전트는 LLM이 먼저 **생각(Reason)**하고, "이 질문에 답변하기 위해 문서 검색(RAG)이 필요한가?" 또는 "그냥 내 지식으로 답해도 되는가?" 등을 판단합니다. 필요할 때만 RAG 함수(Tool)를 호출합니다.


- 상황 판단: RAG 검색 결과가 질문과 관련 없거나 답변에 충분하지 않다고 판단되면, 에이전트는 "문서에서 관련 정보를 찾을 수 없습니다"라고 답변하거나, 추가 정보를 요청하는 등 다른 행동을 취할 수 있습니다.


- 도구 활용: RAG 파이프라인 자체를 하나의 "도구(Tool)"로 간주하여, 에이전트가 계산기, 웹 검색, 다른 API 호출 등 다양한 도구와 함께 상황에 맞게 선택하여 사용할 수 있습니다.

    base_folder_path = "/content/drive/MyDrive/nomu_dataset3" # <<< 원본 데이터 폴더 경로 확인!
    result_dir = "/content/drive/MyDrive/nomu_rag_result"    # <<< 결과 저장 폴더 경로 확인!

# 0: 필요한 라이브러리 설치 (OpenAI 추가)

In [ ]:
# === 단계 0: 필요한 라이브러리 설치 (의존성 충돌 해결 버전 + OpenAI 추가) ===
print("--- 단계 0: 라이브러리 설치 시작 (OpenAI 추가) ---")
!pip install -qU \
    langchain langchain-core langchain-community langchainhub langchain-openai openai \
    pypdf openpyxl xlrd unstructured faiss-cpu sentence-transformers \
    pdf2image pillow pdfminer.six rank_bm25 pillow-heif jq \
    google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread \
    ragas datasets \
    pandas==2.2.2 \
    PyPDF2 \
    fsspec==2025.3.2 # <<< fsspec 버전은 환경에 따라 조정 필요, 원래 버전 사용

# google-ai-generativelanguage는 OpenAI 사용 시 필수는 아님 (필요시 유지)
# !pip install -qU google-ai-generativelanguage==0.6.15

print("\n[알림] 라이브러리 설치/업데이트 완료. langchain-openai, openai 추가됨.")

--- 단계 0: 라이브러리 설치 시작 (OpenAI 추가) ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 149.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/

# 1: 기본 및 필요 라이브러리 임포트 (OpenAI LLM 임포트 추가)

In [ ]:
# === 단계 1: 기본 및 필요 라이브러리 임포트 ===

import os
import glob
import subprocess
import sys
import warnings
import time
import pandas as pd
from google.colab import drive, auth, userdata
import PyPDF2 # 명시적 임포트
import json
import pickle
import torch
import numpy as np
from tqdm.notebook import tqdm
import gspread
from google.auth import default as google_auth_default
from datasets import Dataset
import re
import traceback # 오류 상세 출력을 위해 추가

# LangChain 관련 임포트
from langchain_core.documents import Document
from langchain_community.document_loaders import (
    PyPDFLoader, UnstructuredExcelLoader, CSVLoader,
    UnstructuredFileLoader, DirectoryLoader, GoogleDriveLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings # Google 임베딩 (필요 시 유지)
from langchain_community.embeddings import HuggingFaceEmbeddings # HuggingFace 임베딩 (유지)
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
# === LLM 임포트 변경 ===
# from langchain_google_genai import ChatGoogleGenerativeAI # <<< 삭제 또는 주석 처리
from langchain_openai import ChatOpenAI # <<< OpenAI LLM 클래스 임포트
# =======================
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# RAGAS 관련 임포트 (기존 유지)
try:
    from ragas import evaluate
    from ragas.metrics import context_precision, context_recall, faithfulness, answer_relevancy
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
except ImportError:
    print("!! Ragas 관련 라이브러리가 설치되지 않았습니다. 평가 단계 전에 설치가 필요합니다.")
    LangchainLLMWrapper = None; LangchainEmbeddingsWrapper = None; context_precision = None; context_recall = None; faithfulness = None; answer_relevancy = None

# 기타 평가 관련 임포트 (기존 유지)
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
# import google.generativeai as genai # OpenAI 사용 시 필수는 아님

warnings.filterwarnings("ignore") # 경고 메시지 숨기기

print("--- 단계 1: 라이브러리 임포트 완료 (ChatOpenAI 임포트됨) ---")

--- 단계 1: 라이브러리 임포트 완료 (ChatOpenAI 임포트됨) ---


# 2: 환경 설정 (OpenAI API 키 추가)

    base_folder_path = "/content/drive/MyDrive/nomu_dataset3" # <<< 원본 데이터 폴더 경로 확인!
    result_dir = "/content/drive/MyDrive/nomu_rag_result"    # <<< 결과 저장 폴더 경로 확인!
    # <<< nomu_dataset3 폴더의 실제 ID로 변경하거나 확인! >>>
예)
    https://drive.google.com/drive/u/0/folders/1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U

target_folder_id = "1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U"

In [ ]:
# === 단계 2: 환경 설정 (Drive 마운트, API 키, 경로) ===
print("\n--- 단계 2: 환경 설정 시작 ---")

# --- Google Drive 마운트 (기존 유지) ---
DRIVE_MOUNTED = False
try:
    drive.mount('/content/drive', force_remount=False)
    DRIVE_MOUNTED = True
    print("[성공] Google Drive 마운트 완료.")
except Exception as e:
    print(f"[실패] Google Drive 마운트 오류: {e}")

# --- Google 인증 (Colab 사용자 인증 - 필요시 유지) ---
# Google Drive Loader 등을 사용한다면 인증 유지 필요
try:
    auth.authenticate_user()
    print("[성공] Google Colab 사용자 인증 완료.")
except Exception as e:
    print(f"[실패] Google Colab 인증 오류: {e}")

# === API 키 설정 변경 ===
# --- OpenAI API 키 설정 ---
OPENAI_API_KEY = None
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # Colab Secrets 우선 확인
    if not OPENAI_API_KEY: OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') # 환경 변수 확인
    if not OPENAI_API_KEY: raise ValueError("OpenAI API 키를 Colab Secrets 또는 환경 변수에서 찾을 수 없습니다.")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    print("[성공] OpenAI API 키 로드 및 설정 완료.")
except Exception as e:
    print(f"[실패] OpenAI API 키 로드/설정 오류: {e}")
    print("   !! OpenAI API 키 없이는 이후 LLM, RAGAS 평가 등 사용 불가 !!")

# --- Google AI API 키 설정 (선택 사항) ---
# 만약 Google Embedding 등을 계속 사용한다면 유지, 아니면 제거 가능
# GOOGLE_API_KEY = None
# try:
#     GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
#     if not GOOGLE_API_KEY: GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
#     if GOOGLE_API_KEY:
#         os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#         import google.generativeai as genai
#         genai.configure(api_key=GOOGLE_API_KEY)
#         print("[정보] Google API 키 로드 및 설정 완료 (선택 사항).")
#     # else: print("[정보] Google API 키 로드 안 됨 (선택 사항).") # 키 없어도 오류 아님
# except Exception as e: print(f"[경고] Google API 키 설정 중 오류 발생 (선택 사항): {e}")
# =========================

# --- 경로 설정 (기존 유지) ---
if DRIVE_MOUNTED:
    base_folder_path = "/content/drive/MyDrive/nomu_dataset3"
    result_dir = "/content/drive/MyDrive/nomu_rag_result"
else:
    base_folder_path = "./nomu_data_local"
    result_dir = "./nomu_rag_result_local"

print(f"데이터 소스 검색 경로: {base_folder_path}")
print(f"결과 저장 경로: {result_dir}")
os.makedirs(base_folder_path, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)

# Google Sheets 폴더 ID (필요시 유지)
target_folder_id = "1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U"
print(f"Google Sheets 검색 대상 폴더 ID: {target_folder_id}")

# --- 결과 파일 경로 (기존 유지) ---
vs_status_file = os.path.join(result_dir, "vectorstore_build_status.json")
vs_checkpoint_path = os.path.join(result_dir, "faiss_index_nomu_checkpoint")
vs_final_save_path = os.path.join(result_dir, "faiss_index_nomu_final")
bm25_data_save_path = os.path.join(result_dir, "split_texts_for_bm25.pkl")

print("--- 단계 2: 환경 설정 완료 (OpenAI API 키 설정됨) ---")


--- 단계 2: 환경 설정 시작 ---
Mounted at /content/drive
[성공] Google Drive 마운트 완료.
[성공] Google Colab 사용자 인증 완료.
[성공] OpenAI API 키 로드 및 설정 완료.
데이터 소스 검색 경로: /content/drive/MyDrive/nomu_dataset3
결과 저장 경로: /content/drive/MyDrive/nomu_rag_result
Google Sheets 검색 대상 폴더 ID: 1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U
--- 단계 2: 환경 설정 완료 (OpenAI API 키 설정됨) ---


# 3: 데이터 로딩

In [ ]:
# === 단계 3: 데이터 로딩  ===

print("\n--- 단계 3: 데이터 로딩 시작 ---")

loaded_documents = []
loading_errors = {}

# --- 3.1. Google Sheets 로딩 ---
print("\n--- 3.1 Google Sheets 파일 로딩 ---")
if not target_folder_id or "YOUR_" in target_folder_id:
    print("  ⚠️ 경고: Google Drive 폴더 ID가 유효하지 않아 Google Sheet 로딩 건너<0xEB><0x9A><A9>니다.")
elif not DRIVE_MOUNTED:
     print("  ⚠️ 경고: Google Drive가 마운트되지 않아 Google Sheet 로딩 건너<0xEB><0x9A><A9>니다.")
else:
    try:
        gsheet_loader = GoogleDriveLoader(folder_id=target_folder_id, file_types=["sheet"], recursive=True)
        print(f"  GoogleDriveLoader로 폴더 '{target_folder_id}' 로딩 중...")
        gsheet_docs = gsheet_loader.load()
        if gsheet_docs:
            print(f"  [성공] Google Sheets 로딩 ({len(gsheet_docs)}개 조각).")
            for doc in gsheet_docs:
                doc.metadata['file_type'] = 'google_sheet'
                if 'source' not in doc.metadata and 'id' in doc.metadata:
                    doc.metadata['source'] = f"https://docs.google.com/spreadsheets/d/{doc.metadata['id']}"
            loaded_documents.extend(gsheet_docs)
        else: print("  [정보] 해당 폴더에 Google Sheets 파일 없음.")
    except ImportError as ie: print(f"  ❌ 임포트 오류: {ie}. 관련 라이브러리 설치 필요."); loading_errors['google_sheets'] = str(ie)
    except Exception as e: error_msg = f"{type(e).__name__}: {e}"; print(f"  ❌ 로딩 오류: {error_msg}"); loading_errors['google_sheets'] = error_msg

# --- 3.2. 다른 파일 형식 로딩 (PDF, Excel, CSV, TXT 등) ---
print("\n--- 3.2 다른 파일 형식 로딩 (DirectoryLoader) ---")
if not DRIVE_MOUNTED and not os.path.exists(base_folder_path):
    print(f"  ⚠️ 경고: Drive 미마운트 및 로컬 경로({base_folder_path}) 없음. 파일 로딩 건너<0xEB><0x9A><A9>니다.")
else:
    LOADER_MAPPING = { ".pdf": (PyPDFLoader, {}), ".xlsx": (UnstructuredExcelLoader, {"mode": "single"}), ".xls": (UnstructuredExcelLoader, {"mode": "single"}), ".csv": (CSVLoader, {"encoding": "utf-8"}), ".txt": (UnstructuredFileLoader, {}) }
    supported_extensions = list(LOADER_MAPPING.keys())
    for ext in supported_extensions:
        print(f"\n  '{ext}' 확장자 로딩 ({base_folder_path})...")
        loader_cls, loader_args = LOADER_MAPPING[ext]
        try:
            loader = DirectoryLoader( base_folder_path, glob=f"**/*{ext}", loader_cls=loader_cls, loader_kwargs=loader_args, recursive=True, show_progress=True, use_multithreading=True, silent_errors=False )
            docs = loader.load()
            if docs:
                print(f"    [성공] '{ext}' 로딩 ({len(docs)}개 조각).")
                for doc in docs: doc.metadata['file_type'] = ext.lstrip('.')
                loaded_documents.extend(docs)
            else: print(f"    [정보] '{ext}' 파일 없음.")
        except ImportError as ie: error_msg = f"{ie}"; print(f"    ❌ 임포트 오류: {error_msg}"); loading_errors[f'loader_{ext}'] = f"ImportError: {error_msg}"
        except Exception as e: error_msg = f"{type(e).__name__}: {e}"; print(f"    ❌ 로딩 오류: {error_msg}"); loading_errors[f'loader_{ext}'] = error_msg

# --- 3.3. 로딩 결과 요약 ---
print(f"\n--- 최종 로드된 문서 조각 수: {len(loaded_documents)} ---")
doc_type_counts = {}
for doc in loaded_documents: file_type = doc.metadata.get('file_type', 'unknown'); doc_type_counts[file_type] = doc_type_counts.get(file_type, 0) + 1
print("\n--- 로드된 문서 타입별 개수 ---");
if doc_type_counts: [print(f"  - {f_type}: {count}개 조각") for f_type, count in sorted(doc_type_counts.items())]
else: print("  로드된 문서 없음.")
if loading_errors: print("\n--- 로딩 오류 요약 ---"); [print(f"  - {src}: {err}") for src, err in loading_errors.items()]
if loaded_documents:
    print("\n--- 첫 로드 문서 샘플 ---")
    try: first_doc = loaded_documents[0]; print(f"  타입: {first_doc.metadata.get('file_type')}\n  메타데이터: {first_doc.metadata}\n  내용(200자): {first_doc.page_content[:200]}...")
    except Exception as e: print(f"  !! 샘플 출력 오류: {e}")
else: print("\n로드된 문서 없음. 경로, 파일 형식, 권한 확인 필요.")
print("\n--- 단계 3: 데이터 로딩 완료 ---")



--- 단계 3: 데이터 로딩 시작 ---

--- 3.1 Google Sheets 파일 로딩 ---
  GoogleDriveLoader로 폴더 '1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U' 로딩 중...
  [성공] Google Sheets 로딩 (89개 조각).

--- 3.2 다른 파일 형식 로딩 (DirectoryLoader) ---

  '.pdf' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


100%|██████████| 125/125 [00:20<00:00,  6.17it/s]


    [성공] '.pdf' 로딩 (1748개 조각).

  '.xlsx' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


0it [00:00, ?it/s]


    [정보] '.xlsx' 파일 없음.

  '.xls' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


...

    [정보] '.xls' 파일 없음.

  '.csv' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


0it [00:00, ?it/s]


    [정보] '.csv' 파일 없음.

  '.txt' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


0it [00:00, ?it/s]

    [정보] '.txt' 파일 없음.

--- 최종 로드된 문서 조각 수: 1837 ---

--- 로드된 문서 타입별 개수 ---
  - google_sheet: 89개 조각
  - pdf: 1748개 조각

--- 첫 로드 문서 샘플 ---
  타입: google_sheet
  메타데이터: {'source': 'https://docs.google.com/spreadsheets/d/1QeMvmrcYe6QQ8L1n6o1NQ7ASTPafmzdbuDAAKXFCu3k/edit?gid=0', 'title': 'filtered_qa_dataset - Sheet1', 'row': 1, 'file_type': 'google_sheet'}
  내용(200자): No.: 0
question: 근로계약이 미성년자에게 불리하다고 인정되는 경우 미성년후견인은 그 계약을 해지할 수 있나요?
answer: 네. 근로계약을 해지할 수 있습니다.
ground_truths: ["「근로기준법」 제67조 제2항은 친권자,후견인 또는 고용노동부장관은 근로계약이 미성년자에게 불리하다고 인정하는 경우에는 이를 해지할 수 있다."라고 규정...

--- 단계 3: 데이터 로딩 완료 ---


# 4: 텍스트 분할 (Chunking, 패턴 기반 + 길이 제한)

In [ ]:
# === 단계 4: 텍스트 분할 (패턴 기반 + 길이 제한) ===
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document # Document 클래스 임포트 확인

print("\n--- 단계 4: 텍스트 분할 시작 (패턴 기반 적용) ---")

split_texts = []
chunk_size_setting = 500
chunk_overlap_setting = 100
# 최대 길이 초과 시 사용할 fallback 스플리터
fallback_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size_setting,
    chunk_overlap=chunk_overlap_setting
)

if 'loaded_documents' in locals() and loaded_documents:
    print(f"[정보] 패턴 기반 청킹 시도 (법률 PDF 대상). 기준 크기={chunk_size_setting}, 중첩={chunk_overlap_setting}")
    processed_docs_count = 0
    skipped_docs_count = 0

    for doc in tqdm(loaded_documents, desc="문서 청킹 중"):
        doc_content = doc.page_content
        doc_metadata = doc.metadata
        file_type = doc_metadata.get('file_type', 'unknown')

        # --- PDF 문서(법률 문서로 간주)에만 패턴 기반 적용 ---
        # 파일 타입이 'pdf'가 아니거나 내용이 없으면 기본 스플리터 사용
        if file_type != 'pdf' or not doc_content.strip():
            if doc_content.strip(): # 내용이 있을 때만 분할
                 try:
                     sub_chunks = fallback_splitter.split_text(doc_content)
                     for chunk_text in sub_chunks:
                         split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                     processed_docs_count += 1
                 except Exception as e:
                      print(f"\n!! 기본 분할 오류 (타입: {file_type}, 소스: {doc_metadata.get('source', 'N/A')}): {e}")
                      skipped_docs_count += 1
            else:
                skipped_docs_count += 1
            continue # 다음 문서로 이동

        # --- 패턴 기반 청킹 로직 (PDF 대상) ---
        try:
            # 1. "제X조" 패턴으로 먼저 크게 나누기
            # 정규표현식: '제' + (선택적 공백) + 숫자 + (선택적 공백) + '조'
            # re.split은 구분자도 결과에 포함시키므로, 이를 활용하여 조 제목을 유지
            preliminary_chunks_with_title = re.split(r'(제\s?\d+\s?조)', doc_content)

            current_chunk_content = ""
            # 첫 부분 처리 (첫 '제X조' 이전 내용)
            if preliminary_chunks_with_title[0].strip():
                 current_chunk_content = preliminary_chunks_with_title[0].strip()

            # '제X조' 제목과 그 내용을 묶어서 처리
            for i in range(1, len(preliminary_chunks_with_title), 2):
                title = preliminary_chunks_with_title[i] # 예: "제1조"
                content = preliminary_chunks_with_title[i+1] if (i+1) < len(preliminary_chunks_with_title) else ""

                article_block = title + content # "제1조 내용..."

                # 이전 청크가 있고 + 현재 조항을 합쳐도 크기를 넘지 않으면 합침
                # (짧은 조항들이 합쳐지는 효과)
                if current_chunk_content and (len(current_chunk_content) + len(article_block) <= chunk_size_setting):
                    current_chunk_content += "\n\n" + article_block # 문단 구분 추가
                else:
                    # 이전 청크가 너무 길었거나, 합치면 길어지는 경우
                    # 이전 청크를 최종 처리하고 현재 조항으로 새 청크 시작
                    if current_chunk_content: # 이전 청크 내용이 있으면
                        # 이전 청크가 최대 크기를 넘는지 확인
                        if len(current_chunk_content) > chunk_size_setting:
                            # 넘으면 fallback 스플리터로 재분할
                            sub_chunks = fallback_splitter.split_text(current_chunk_content)
                            for chunk_text in sub_chunks:
                                split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                        else:
                            # 안 넘으면 그대로 추가
                            split_texts.append(Document(page_content=current_chunk_content, metadata=doc_metadata.copy()))

                    # 현재 조항으로 새 청크 시작
                    current_chunk_content = article_block.strip() # 새 청크 시작

            # 마지막 남은 청크 처리
            if current_chunk_content:
                 if len(current_chunk_content) > chunk_size_setting:
                     sub_chunks = fallback_splitter.split_text(current_chunk_content)
                     for chunk_text in sub_chunks:
                         split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                 else:
                     split_texts.append(Document(page_content=current_chunk_content, metadata=doc_metadata.copy()))

            processed_docs_count += 1

        except Exception as e:
            print(f"\n!! 패턴 기반 분할 오류 (소스: {doc_metadata.get('source', 'N/A')}): {e}")
            # 오류 발생 시 해당 문서는 기본 스플리터로 처리 시도 (선택 사항)
            try:
                sub_chunks = fallback_splitter.split_text(doc_content)
                for chunk_text in sub_chunks:
                     split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                print(f"  -> 오류 발생하여 기본 분할 방식으로 처리함.")
                processed_docs_count += 1
            except Exception as fallback_e:
                 print(f"  -> 기본 분할 방식도 실패: {fallback_e}")
                 skipped_docs_count += 1

    print(f"\n[성공] 총 {processed_docs_count}개 원본 조각 처리 -> {len(split_texts)}개 청크 분할 완료.")
    if skipped_docs_count > 0: print(f"[경고] {skipped_docs_count}개 원본 조각 처리 중 오류 발생하여 건너<0xEB><0x9B><0x81>.")

    if split_texts:
         print("\n첫 분할 청크 샘플:")
         print(f"  내용(200자): {split_texts[0].page_content[:200]}...")
         print(f"  메타데이터: {split_texts[0].metadata}")
         # 크기 확인 (Fallback 적용 후에도 클 수 있음)
         oversized = [len(c.page_content) for c in split_texts if len(c.page_content) > chunk_size_setting + chunk_overlap_setting] # 오버랩 고려
         if oversized: print(f"\n[경고] {len(oversized)}개 청크가 최대 크기({chunk_size_setting}자)를 약간 초과할 수 있음 (최대 {max(oversized)}자).")
         else: print(f"\n[정보] 모든 청크 크기 비교적 정상.")
    else: print("[정보] 생성된 청크 없음.")

else:
    print("!! 분할할 로드된 문서 없음.")

print("--- 단계 4: 텍스트 분할 완료 ---")


--- 단계 4: 텍스트 분할 시작 (패턴 기반 적용) ---
[정보] 패턴 기반 청킹 시도 (법률 PDF 대상). 기준 크기=500, 중첩=100


문서 청킹 중:   0%|          | 0/1837 [00:00<?, ?it/s]


[성공] 총 1837개 원본 조각 처리 -> 9429개 청크 분할 완료.

첫 분할 청크 샘플:
  내용(200자): No.: 0
question: 근로계약이 미성년자에게 불리하다고 인정되는 경우 미성년후견인은 그 계약을 해지할 수 있나요?
answer: 네. 근로계약을 해지할 수 있습니다.
ground_truths: ["「근로기준법」 제67조 제2항은 친권자,후견인 또는 고용노동부장관은 근로계약이 미성년자에게 불리하다고 인정하는 경우에는 이를 해지할 수 있다."라고 규정...
  메타데이터: {'source': 'https://docs.google.com/spreadsheets/d/1QeMvmrcYe6QQ8L1n6o1NQ7ASTPafmzdbuDAAKXFCu3k/edit?gid=0', 'title': 'filtered_qa_dataset - Sheet1', 'row': 1, 'file_type': 'google_sheet'}

[정보] 모든 청크 크기 비교적 정상.
--- 단계 4: 텍스트 분할 완료 ---


# 5: 임베딩 모델 설정 (text-embedding-3-large)

In [ ]:
# === 단계 5: 임베딩 모델 설정 (OpenAI: text-embedding-3-large) ===
print("\n--- 단계 5: 임베딩 모델 설정 시작 (OpenAI: text-embedding-3-large) ---")

# 필요한 라이브러리 임포트
try:
    # OpenAI 임베딩을 위한 클래스 임포트
    # pip install -U langchain-openai  <-- 먼저 이 패키지를 설치해야 합니다.
    from langchain_openai import OpenAIEmbeddings
    print("[정보] langchain_openai.OpenAIEmbeddings 임포트 확인.")
except ImportError as e:
    print(f"!! [오류] 필요한 라이브러리 임포트 실패: {e}")
    print("   langchain-openai, openai 패키지가 설치되었는지 확인하세요.")
    # 필요한 클래스가 없으면 이후 진행 불가
    OpenAIEmbeddings = None

embeddings = None # 최종 임베딩 객체 변수 초기화

if OpenAIEmbeddings: # 라이브러리 임포트 성공 시 진행
    try:
        # 사용할 OpenAI 모델 지정
        openai_model_name = "text-embedding-3-large"
        print(f"[정보] OpenAI 임베딩 모델 ({openai_model_name}) 설정 시도...")
        print("[정보] 단계 2에서 설정된 OPENAI_API_KEY 환경 변수를 사용합니다.")

        # <<< OpenAIEmbeddings 클래스 사용 (API 키 자동 감지) >>>
        # 단계 2에서 os.environ["OPENAI_API_KEY"] 가 설정되었으므로,
        # API 키를 명시적으로 전달할 필요 없이 자동으로 사용됩니다.
        embeddings = OpenAIEmbeddings(
            model=openai_model_name # 사용할 모델 이름만 지정
            # openai_api_key 파라미터 생략
        )
        # ------------------------------------------------------

        # 간단한 테스트 (API 키 유효성 및 통신 확인)
        print("[정보] 임베딩 모델 테스트 중 (OpenAI API 호출)...")
        _ = embeddings.embed_query("테스트 문장입니다.")
        print(f"[성공] OpenAI 임베딩 모델 ({openai_model_name}) 설정 및 테스트 완료.")

    except Exception as e:
        print(f"!! [오류] OpenAI 임베딩 모델 ({openai_model_name}) 설정 실패: {e}")
        print("   - 단계 2에서 OpenAI API 키가 올바르게 설정되었는지 확인하세요.") # <<< 오류 메시지 수정
        print("   - OpenAI API 키 자체의 유효성 및 할당량(quota)을 확인하세요.")
        print("   - 인터넷 연결 상태 및 OpenAI 서비스 상태를 확인하세요.")
        print("   - 관련 라이브러리(langchain-openai, openai) 설치 및 호환성을 확인하세요.")
        embeddings = None # 실패 시 None으로 설정
else:
    print("!! 필요한 라이브러리(OpenAIEmbeddings) 임포트 실패. 임베딩 모델 설정 불가.")

print("--- 단계 5: 임베딩 모델 설정 완료 ---")

# OpenAI 모델은 외부 API를 사용하므로 로컬 GPU/CPU 확인은 불필요합니다.


--- 단계 5: 임베딩 모델 설정 시작 (OpenAI: text-embedding-3-large) ---
[정보] langchain_openai.OpenAIEmbeddings 임포트 확인.
[정보] OpenAI 임베딩 모델 (text-embedding-3-large) 설정 시도...
[정보] 단계 2에서 설정된 OPENAI_API_KEY 환경 변수를 사용합니다.
[정보] 임베딩 모델 테스트 중 (OpenAI API 호출)...
[성공] OpenAI 임베딩 모델 (text-embedding-3-large) 설정 및 테스트 완료.
--- 단계 5: 임베딩 모델 설정 완료 ---


# 단계 6: Vector Store 구축 (FAISS)

--- 6.2: 경로 설정 ---
📂 결과 저장 경로: /content/drive/MyDrive/nomu_rag_result
  - 상태 파일: /content/drive/MyDrive/nomu_rag_result/vectorstore_build_status.json
  - 체크포인트 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_checkpoint
  - 최종 인덱스 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_final
  - BM25용 데이터 파일: /content/drive/MyDrive/nomu_rag_result/split_texts_for_bm25.pkl

In [ ]:
# -*- coding: utf-8 -*-
# === 단계 6: Vector Store 구축 (FAISS) 및 BM25용 데이터 저장 ===
# 체크포인팅 방식을 save_local로 변경

import time
import os
import json
# import pickle # pickle은 BM25 데이터 저장에만 사용
import torch
import traceback
from tqdm.auto import tqdm
# LangChain 관련 임포트 확인
if 'FAISS' not in locals() or 'Document' not in locals():
    from langchain_community.vectorstores import FAISS
    from langchain_core.documents import Document
    print("⚠️ FAISS 또는 Document 클래스 재임포트됨.")
if 'HuggingFaceEmbeddings' not in locals() and 'GoogleGenerativeAIEmbeddings' not in locals():
    # 사용 중인 임베딩 클래스를 임포트해야 합니다.
    from langchain_community.embeddings import HuggingFaceEmbeddings # 예시
    # from langchain_google_genai import GoogleGenerativeAIEmbeddings # 예시
    print("⚠️ 임베딩 클래스 재임포트됨.")


print("\n--- 단계 6: Vector Store 구축 및 BM25 데이터 저장 시작 ---")

# --- 6.1: 설정 및 입력 변수 확인 ---
print("\n--- 6.1: 설정 및 입력 변수 확인 ---")
vectorstore = None
batch_size = 500 # 배치 크기 확인
sleep_time = 5
max_retries = 3
total_chunks = 0
start_index = 0
loaded_from_checkpoint = False

if 'split_texts' not in locals() or not isinstance(split_texts, list) or not split_texts: print("!! 오류: 'split_texts' 없음. 중단."); exit()
elif 'embeddings' not in locals() or embeddings is None: print("!! 오류: 'embeddings' 없음. 중단."); exit()
else: total_chunks = len(split_texts); print(f"✅ 입력 확인: 총 {total_chunks}개 청크 및 임베딩 모델 확인됨.")

# --- 6.2: 경로 설정 및 디렉토리 생성 ---
print("\n--- 6.2: 경로 설정 ---")
if 'result_dir' not in locals() or not result_dir: result_dir = "/content/drive/MyDrive/nomu_rag_result"; print(f"⚠️ result_dir 변수 없어 기본 경로 설정: {result_dir}"); os.makedirs(result_dir, exist_ok=True)
else: print(f"📂 결과 저장 경로: {result_dir}"); os.makedirs(result_dir, exist_ok=True)

vs_status_file = os.path.join(result_dir, "vectorstore_build_status.json")
# <<< 체크포인트 경로를 폴더로 변경 (save_local 사용) >>>
vs_checkpoint_path = os.path.join(result_dir, "faiss_index_nomu_checkpoint") # .pkl 대신 폴더명
vs_final_save_path = os.path.join(result_dir, "faiss_index_nomu_final")
bm25_data_save_path = os.path.join(result_dir, "split_texts_for_bm25.pkl")

print(f"  - 상태 파일: {vs_status_file}")
print(f"  - 체크포인트 폴더 (FAISS): {vs_checkpoint_path}") # <<< 이름 변경
print(f"  - 최종 인덱스 폴더 (FAISS): {vs_final_save_path}")
print(f"  - BM25용 데이터 파일: {bm25_data_save_path}")

# --- 6.3: GPU 확인 --- (이전과 동일)
print("\n--- 6.3: 장치 확인 ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'; print(f"ℹ️ 사용할 장치: {device}");
if device == 'cpu': print("⚠️ 경고: GPU 사용 불가. 시간 소요 예상.")

# --- 6.4: 이전 작업 상태 및 체크포인트 로드 ---
if total_chunks > 0:
    print("\n--- 6.4: 이전 작업 상태 및 체크포인트 로드 ---")
    try:
        # <<< 체크포인트 파일 대신 폴더 존재 여부 확인 >>>
        if os.path.exists(vs_status_file) and os.path.isdir(vs_checkpoint_path):
            with open(vs_status_file, 'r') as f_status: status_data = json.load(f_status); last_processed_index = status_data.get('last_processed_index', -1); start_index = last_processed_index + 1; print(f"  - 상태 로드 완료. 마지막 인덱스: {last_processed_index}")

            # <<< pickle.load 대신 FAISS.load_local 사용 >>>
            print(f"  - 체크포인트 인덱스 로딩: {vs_checkpoint_path}")
            if 'embeddings' in locals() and embeddings:
                vectorstore = FAISS.load_local(
                    folder_path=vs_checkpoint_path,
                    embeddings=embeddings,
                    allow_dangerous_deserialization=True # 신뢰할 수 있는 소스일 때만
                )
                print(f"  - FAISS 체크포인트 로드 완료. {vectorstore.index.ntotal} 벡터 포함.")
                loaded_from_checkpoint = True
            else:
                print("  !! 오류: 임베딩 함수('embeddings')가 없어 체크포인트를 로드할 수 없음. 처음부터 시작.")
                vectorstore = None; start_index = 0; loaded_from_checkpoint = False

            if start_index >= total_chunks: print("  ✅ 이전 FAISS 작업 완료됨.")
            elif loaded_from_checkpoint: print(f"  ▶️ {start_index}번 인덱스부터 FAISS 작업 재개.")
            # else 블록은 위에서 처리됨

        else: print(f"  - 이전 상태 파일 또는 체크포인트 폴더 없음. 처음부터 시작."); vectorstore = None; start_index = 0
    except Exception as e: print(f"  ⚠️ 상태/체크포인트 로드 오류: {e}. 처음부터 시작."); traceback.print_exc(); vectorstore = None; start_index = 0
else: print("ℹ️ 처리할 청크 없음.")

# --- 6.5: FAISS Vector Store 구축 ---
if total_chunks > 0 and start_index < total_chunks:
    print(f"\n--- 6.5: FAISS 인덱스 구축 시작 (총 {total_chunks} 중 {start_index}부터, 배치 {batch_size}) ---")
    all_processed_successfully = True
    try:
        progress_bar = tqdm(range(start_index, total_chunks, batch_size), initial=start_index // batch_size, total=-(total_chunks // -batch_size), desc="FAISS 구축 중")
        for i in progress_bar:
            batch_start_idx = i; batch_end_idx = min(i + batch_size, total_chunks); batch_docs = split_texts[batch_start_idx:batch_end_idx]
            for attempt in range(max_retries):
                try:
                    if i == start_index and not loaded_from_checkpoint:
                        progress_bar.set_description(f"첫 배치({batch_start_idx}-{batch_end_idx-1}) 생성 중")
                        vectorstore = FAISS.from_documents(batch_docs, embeddings)
                        print(f"\n   - 첫 배치 FAISS 생성 완료 (벡터 {vectorstore.index.ntotal}개)")
                    elif vectorstore is not None:
                        progress_bar.set_description(f"배치({batch_start_idx}-{batch_end_idx-1}) 추가 중")
                        vectorstore.add_documents(batch_docs)
                        print(f"\n   - 배치 FAISS 추가 완료 (총 벡터 {vectorstore.index.ntotal}개)")
                    else: raise ValueError("Vectorstore 객체 None 상태.")

                    current_processed_index = batch_end_idx - 1
                    try:
                        # <<< 체크포인팅: 상태 저장 후 save_local 호출 >>>
                        with open(vs_status_file, 'w') as f_status: json.dump({'last_processed_index': current_processed_index}, f_status)
                        # 체크포인트 폴더에 덮어쓰기 (이전 파일 삭제됨)
                        vectorstore.save_local(vs_checkpoint_path)
                        # print(f"  💾 체크포인트 저장 완료 (폴더: {vs_checkpoint_path}, 인덱스: {current_processed_index})")
                    except Exception as e_save: print(f"  ⚠️ 진행 상황(체크포인트) 저장 실패: {e_save}")
                    break # 성공 시 재시도 루프 탈출
                except Exception as e:
                    print(f"\n!! 배치 처리 오류 (시도 {attempt+1}/{max_retries}): {type(e).__name__} - {e}")
                    traceback.print_exc()
                    if attempt < max_retries - 1: wait_time = sleep_time * (attempt + 2); progress_bar.set_description(f"오류, {wait_time}초 후 재시도..."); print(f"  ... {wait_time}초 후 재시도 ..."); time.sleep(wait_time)
                    else: print("!! 최대 재시도 초과. 구축 중단."); all_processed_successfully = False; raise RuntimeError(f"최대 재시도 실패: {e}")
            if not all_processed_successfully: break
            if batch_end_idx < total_chunks: time.sleep(sleep_time) # API Rate Limit

        # --- 루프 완료 후 최종 처리 ---
        if all_processed_successfully:
            print("\n[성공] 모든 FAISS 배치 처리 완료.")
            # --- 6.6: 최종 결과 저장 ---
            print("\n--- 6.6: 최종 결과 저장 ---")
            # 1. 최종 FAISS 인덱스 저장 (save_local)
            print(f"💾 최종 FAISS 인덱스 저장: {vs_final_save_path}")
            try:
                if vectorstore: vectorstore.save_local(vs_final_save_path); print(f"✅ 최종 FAISS 인덱스 저장 완료.")
                else: print("⚠️ 최종 저장 시 VectorStore 객체 없음.")
            except Exception as e: print(f"❌ 최종 FAISS 인덱스 저장 오류: {e}"); traceback.print_exc()
            # 2. BM25용 데이터 저장 (pickle)
            print(f"💾 BM25용 텍스트 데이터 저장: {bm25_data_save_path}")
            try:
                if 'split_texts' in locals() and split_texts:
                    # <<< pickle 임포트 확인 >>>
                    import pickle
                    with open(bm25_data_save_path, 'wb') as f_texts: pickle.dump(split_texts, f_texts); print(f"✅ BM25용 텍스트 데이터 ({len(split_texts)}개 청크) 저장 완료.")
                else: print("⚠️ BM25용 데이터('split_texts') 없어 저장 불가.")
            except Exception as e: print(f"❌ split_texts 저장 실패: {e}"); traceback.print_exc()
            # (선택) 중간 파일 삭제
            # try: ... (삭제 로직) ... except ...

    except Exception as e_main_loop: print(f"\n!! FAISS 구축 실패: {e_main_loop}"); print(f"[정보] 마지막 성공 지점 데이터는 '{vs_checkpoint_path}' 폴더에 있을 수 있음.")

# --- 이전 실행에서 이미 완료된 경우 처리 ---
elif total_chunks > 0 and start_index >= total_chunks:
     print("\n✅ FAISS Vector Store 구축 작업이 이미 완료된 상태입니다.")
     # 완료된 상태에서도 최종 파일들이 존재하는지 확인하고 없으면 저장 시도
     # 1. 최종 FAISS 인덱스 확인 및 저장
     if not os.path.isdir(vs_final_save_path): # <<< 폴더 존재 여부 확인 >>>
         if vectorstore: # 체크포인트에서 로드된 vectorstore가 있다면
             print(f"\n💾 최종 FAISS 인덱스 폴더 없어 저장 시도: '{vs_final_save_path}'") # <<< print 문 분리
             # <<< try...except 블록을 새 줄에서 시작 >>>
             try:
                 vectorstore.save_local(vs_final_save_path)
                 print("✅ 저장 완료.")
             except Exception as e:
                 print(f"❌ 저장 오류: {e}")
                 traceback.print_exc()
         else:
             print(f"⚠️ 최종 FAISS 인덱스 폴더 없고, 로드된 vectorstore도 없어 저장 불가.")

     # 2. BM25용 데이터 확인 및 저장
     if not os.path.exists(bm25_data_save_path):
         if 'split_texts' in locals() and split_texts:
             print(f"\n💾 BM25용 텍스트 데이터 파일({bm25_data_save_path}) 없어 저장 시도...") # <<< print 문 분리
             # <<< try...except 블록을 새 줄에서 시작 >>>
             try:
                 # <<< pickle 임포트 확인 (필요시) >>>
                 import pickle
                 with open(bm25_data_save_path, 'wb') as f:
                     pickle.dump(split_texts, f)
                 print("✅ 저장 완료.")
             except Exception as e:
                 print(f"❌ 저장 실패: {e}")
                 traceback.print_exc()
         else:
             print(f"⚠️ BM25용 데이터 파일 없고, split_texts 변수도 없어 저장 불가.")

print("\n--- 단계 6: Vector Store 처리 및 BM25 데이터 저장 완료 ---")


--- 단계 6: Vector Store 구축 및 BM25 데이터 저장 시작 ---

--- 6.1: 설정 및 입력 변수 확인 ---
✅ 입력 확인: 총 9429개 청크 및 임베딩 모델 확인됨.

--- 6.2: 경로 설정 ---
📂 결과 저장 경로: /content/drive/MyDrive/nomu_rag_result
  - 상태 파일: /content/drive/MyDrive/nomu_rag_result/vectorstore_build_status.json
  - 체크포인트 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_checkpoint
  - 최종 인덱스 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_final
  - BM25용 데이터 파일: /content/drive/MyDrive/nomu_rag_result/split_texts_for_bm25.pkl

--- 6.3: 장치 확인 ---
ℹ️ 사용할 장치: cpu
⚠️ 경고: GPU 사용 불가. 시간 소요 예상.

--- 6.4: 이전 작업 상태 및 체크포인트 로드 ---
  - 이전 상태 파일 또는 체크포인트 폴더 없음. 처음부터 시작.

--- 6.5: FAISS 인덱스 구축 시작 (총 9429 중 0부터, 배치 500) ---


FAISS 구축 중:   0%|          | 0/19 [00:00<?, ?it/s]


   - 첫 배치 FAISS 생성 완료 (벡터 500개)

   - 배치 FAISS 추가 완료 (총 벡터 1000개)

   - 배치 FAISS 추가 완료 (총 벡터 1500개)

   - 배치 FAISS 추가 완료 (총 벡터 2000개)

   - 배치 FAISS 추가 완료 (총 벡터 2500개)

   - 배치 FAISS 추가 완료 (총 벡터 3000개)

   - 배치 FAISS 추가 완료 (총 벡터 3500개)

   - 배치 FAISS 추가 완료 (총 벡터 4000개)

   - 배치 FAISS 추가 완료 (총 벡터 4500개)

   - 배치 FAISS 추가 완료 (총 벡터 5000개)

   - 배치 FAISS 추가 완료 (총 벡터 5500개)

   - 배치 FAISS 추가 완료 (총 벡터 6000개)

   - 배치 FAISS 추가 완료 (총 벡터 6500개)

   - 배치 FAISS 추가 완료 (총 벡터 7000개)

   - 배치 FAISS 추가 완료 (총 벡터 7500개)

   - 배치 FAISS 추가 완료 (총 벡터 8000개)

   - 배치 FAISS 추가 완료 (총 벡터 8500개)

   - 배치 FAISS 추가 완료 (총 벡터 9000개)

   - 배치 FAISS 추가 완료 (총 벡터 9429개)

[성공] 모든 FAISS 배치 처리 완료.

--- 6.6: 최종 결과 저장 ---
💾 최종 FAISS 인덱스 저장: /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_final
✅ 최종 FAISS 인덱스 저장 완료.
💾 BM25용 텍스트 데이터 저장: /content/drive/MyDrive/nomu_rag_result/split_texts_for_bm25.pkl
✅ BM25용 텍스트 데이터 (9429개 청크) 저장 완료.

--- 단계 6: Vector Store 처리 및 BM25 데이터 저장 완료 ---


# 단계 8: Retriever 설정 (하이브리드 검색)

In [ ]:
# === 단계 8: Retriever 설정 (하이브리드 검색) ===

print("\n--- 단계 8: Retriever 설정 시작 (하이브리드 검색) ---")

# 사용할 VectorStore와 split_texts 결정
# 로드 성공 시 로드된 것 사용, 아니면 구축 단계에서 생성된 것 사용
final_vectorstore = None
final_split_texts = None # BM25용

if 'loaded_vectorstore' in locals() and loaded_vectorstore:
    final_vectorstore = loaded_vectorstore
    print("[정보] 로드된 FAISS VectorStore 사용.")
elif 'vectorstore' in locals() and vectorstore:
    final_vectorstore = vectorstore
    print("[정보] 새로 생성된 FAISS VectorStore 사용.")
else:
    print("!! 오류: 사용 가능한 FAISS VectorStore 객체 없음.")

if 'loaded_split_texts' in locals() and loaded_split_texts:
    final_split_texts = loaded_split_texts
    print("[정보] 로드된 split_texts 데이터 사용 (BM25용).")
elif 'split_texts' in locals() and split_texts:
    final_split_texts = split_texts
    print("[정보] 현재 세션의 split_texts 데이터 사용 (BM25용).")
else:
    print("!! 오류: BM25용 split_texts 데이터 없음.")

retriever = None # 최종 리트리버 초기화

if final_vectorstore:
    # Dense Retriever (FAISS)
    faiss_retriever = final_vectorstore.as_retriever(search_kwargs={'k': 6})
    print(f"- Dense Retriever (FAISS) 설정 완료 (k={faiss_retriever.search_kwargs.get('k')}).")

    if final_split_texts: # BM25용 데이터가 있을 때만 하이브리드 시도
        try:
            # Sparse Retriever (BM25)
            bm25_retriever = BM25Retriever.from_documents(final_split_texts)
            bm25_retriever.k = 6
            print(f"- Sparse Retriever (BM25) 설정 완료 (k={bm25_retriever.k}).")
            # Ensemble Retriever
            ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], weights=[0.4, 0.6])
            retriever = ensemble_retriever
            print(f"- Ensemble Retriever 설정 완료 (Weights: BM25=0.7, FAISS=0.3).")
        except Exception as e:
            print(f"!! BM25/Ensemble 설정 실패: {e}. Dense Retriever만 사용.")
            retriever = faiss_retriever # Fallback
    else:
        print("⚠️ BM25 데이터 없어 Dense Retriever(FAISS)만 사용합니다.")
        retriever = faiss_retriever # Fallback
else:
    print("!! Vector Store 준비 안 됨. Retriever 설정 불가.")

print("--- 단계 8: Retriever 설정 완료 ---")



--- 단계 8: Retriever 설정 시작 (하이브리드 검색) ---
[정보] 새로 생성된 FAISS VectorStore 사용.
[정보] 현재 세션의 split_texts 데이터 사용 (BM25용).
- Dense Retriever (FAISS) 설정 완료 (k=6).
- Sparse Retriever (BM25) 설정 완료 (k=6).
- Ensemble Retriever 설정 완료 (Weights: BM25=0.7, FAISS=0.3).
--- 단계 8: Retriever 설정 완료 ---


# 9: LLM 설정 (OpenAI: gpt-4.1-nano)

현재의 rag 파이프라인은 RAGAS 성능 평가용 LLM과 응답 생성용 LLM을 분리하지 않고 동일하게 사용하는 코드입니다.

In [ ]:
# === 단계 9: LLM 설정 (OpenAI - gpt-4.1-nano) ===

print("\n--- 단계 9: LLM 설정 시작 (OpenAI) ---")
llm = None
# OpenAI API 키 설정 여부 확인 (단계 2에서 설정됨)
if 'OPENAI_API_KEY' in os.environ and os.environ["OPENAI_API_KEY"]:
    try:
        # === 사용할 OpenAI 모델 지정 ===
        # llm_model_name = "gpt-4o" # 최신 모델 (성능과 속도 균형)
        llm_model_name = "gpt-4.1-nano" # 만약 더 작고 빠른 모델이 필요하다면 고려 (현재는 gpt-4o가 가장 유사)
        # llm_model_name = "gpt-3.5-turbo" # 속도/비용 우선 시 고려
        # ==============================

        llm = ChatOpenAI(
            model=llm_model_name,
            temperature=0 # 답변의 창의성 조절 (낮을수록 결정적)
            # max_tokens=1024 # 필요시 최대 출력 토큰 수 제한
        )
        print(f"[성공] OpenAI ({llm.model_name}) LLM 로딩 완료.") # .model 대신 .model_name 사용

    except ImportError:
         print("!! [오류] langchain-openai 또는 openai 라이브러리가 설치되지 않았습니다.")
         print("   단계 0의 설치 명령을 확인하고 런타임을 재시작하세요.")
    except Exception as e:
        print(f"!! [오류] OpenAI LLM 로딩 실패: {e}")
        traceback.print_exc() # 상세 오류 출력
else:
    print("!! [오류] OpenAI API 키가 설정되지 않았습니다. LLM 로드 불가.")

print("--- 단계 9: LLM 설정 완료 ---")


--- 단계 9: LLM 설정 시작 (OpenAI) ---
[성공] OpenAI (gpt-4.1-nano) LLM 로딩 완료.
--- 단계 9: LLM 설정 완료 ---


# 10: RAG Chain/파이프라인 구축

In [ ]:
# === 단계 10: RAG Chain/파이프라인 구축 ===
# (코드 변경 없음 - 단계 9에서 변경된 llm 객체를 사용)
print("\n--- 단계 10: RAG Chain 구축 시작 ---")
qa_chain = None
if llm and retriever: # llm이 ChatOpenAI 인스턴스로 준비됨
    template = """당신은 한국의 노무 규정 및 관련 문서에 기반하여 질문에 답변하는 유용한 AI 어시스턴트입니다.
    주어진 다음 컨텍스트 정보만을 사용하여 질문에 답변하십시오. 컨텍스트에 없는 내용은 답변에 포함하지 마십시오.
    만약 컨텍스트 정보만으로 질문에 답할 수 없다면, "
    제공된 문서 내용만으로는 답변할 수 없습니다."라고 명확히 답변하십시오.
    답변은 간결하고 명확하게 한국어로 작성해주세요.

    컨텍스트:
    {context}

    질문: {question}

    답변 (한국어):"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
    try:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm, # ChatOpenAI 인스턴스 사용
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )
        print("[성공] RetrievalQA Chain 구축 완료.")
    except Exception as e: print(f"!! RAG Chain 구축 실패: {e}")
else: print("!! LLM 또는 Retriever 준비 안 됨. RAG Chain 구축 불가.")
print("--- 단계 10: RAG Chain 구축 완료 ---")


--- 단계 10: RAG Chain 구축 시작 ---
[성공] RetrievalQA Chain 구축 완료.
--- 단계 10: RAG Chain 구축 완료 ---


# 10.1: RAG 파이프라인을 Tool로 정의

In [ ]:
# === 단계 10.1: RAG 파이프라인을 Tool로 정의 ===
print("\n--- 단계 10.1: RAG 파이프라인을 Tool로 정의 시작 ---")

from langchain.tools import Tool

rag_tool = None

# 단계 10에서 qa_chain이 성공적으로 생성되었는지 확인
if 'qa_chain' in locals() and qa_chain is not None:
    try:
        # Tool 정의
        rag_tool = Tool.from_function(
            func=lambda q: qa_chain.invoke({"query": q}), # qa_chain을 호출하는 함수
            name="KoreanLaborLawQASystem", # 도구 이름 (간결하고 명확하게)
            description="""한국의 노무 법률, 규정, 관련 판례 및 문서에 대한 구체적인 질문에 답변할 때 사용합니다.
                        근로기준법, 해고 절차, 임금, 휴가, 산업 안전 등 노무 관련 질문이 입력되었을 때 이 도구를 사용해야 합니다.
                        일반적인 상식이나 대화에는 사용하지 마세요. 입력은 사용자의 질문이어야 합니다.""", # <<< LLM이 이해할 수 있도록 상세히 작성!
            return_direct=False # False로 설정해야 LLM이 도구 결과를 보고 추가 생각을 할 수 있음 (ReAct 방식)
                               # True로 하면 도구 결과가 바로 최종 답변이 됨
        )
        print("[성공] RAG 파이프라인을 위한 Tool 정의 완료.")
        print(f"  Tool Name: {rag_tool.name}")
        print(f"  Tool Description: {rag_tool.description}")

    except Exception as e:
        print(f"!! [오류] Tool 정의 중 오류 발생: {e}")
        traceback.print_exc()
else:
    print("!! [오류] 'qa_chain'이 정의되지 않았습니다. Tool을 만들 수 없습니다.")

print("--- 단계 10.1: Tool 정의 완료 ---")


--- 단계 10.1: RAG 파이프라인을 Tool로 정의 시작 ---
[성공] RAG 파이프라인을 위한 Tool 정의 완료.
  Tool Name: KoreanLaborLawQASystem
  Tool Description: 한국의 노무 법률, 규정, 관련 판례 및 문서에 대한 구체적인 질문에 답변할 때 사용합니다.
                        근로기준법, 해고 절차, 임금, 휴가, 산업 안전 등 노무 관련 질문이 입력되었을 때 이 도구를 사용해야 합니다.
                        일반적인 상식이나 대화에는 사용하지 마세요. 입력은 사용자의 질문이어야 합니다.
--- 단계 10.1: Tool 정의 완료 ---


# 10.2: ReAct 에이전트 설정 (OpenAI Tools Agent)

In [ ]:
# === 단계 10.2: ReAct 에이전트 설정 (OpenAI Tools Agent) ===
print("\n--- 단계 10.2: ReAct 에이전트 설정 시작 ---")

from langchain import hub # ReAct 프롬프트 템플릿 가져오기 위함
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent_executor = None
agent = None

# 단계 9에서 LLM(ChatOpenAI)이 성공적으로 로드되었는지 확인
# 단계 10.1에서 rag_tool이 성공적으로 생성되었는지 확인
if 'llm' in locals() and llm is not None and 'rag_tool' in locals() and rag_tool is not None:
    try:
        # 사용할 도구 리스트 정의 (현재는 RAG 도구 하나)
        tools = [rag_tool]
        print(f"[정보] 에이전트가 사용할 도구: {[tool.name for tool in tools]}")

        # ReAct 스타일의 프롬프트 가져오기 (OpenAI Tools Agent용)
        # LangChain Hub에서 검증된 프롬프트를 사용하는 것이 좋음
        # prompt = hub.pull("hwchase17/openai-tools-agent") # 예시 프롬프트 주소
        # 또는 직접 ChatPromptTemplate 구성
        from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

        # 간단한 ReAct 스타일 프롬프트 예시 (필요시 수정)
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", """당신은 질문에 답변하는 AI 어시스턴트입니다.
             필요하다면 다음 도구를 사용할 수 있습니다. 질문을 분석하고, 도구 사용이 필요하다고 판단되면 도구를 사용하세요.
             만약 도구 없이 답변할 수 있거나, 질문이 도구 사용에 적합하지 않다면 직접 답변하세요.
             답변은 항상 한국어로 작성해주세요."""),
            MessagesPlaceholder(variable_name="chat_history", optional=True), # 대화 기록 (필요시)
            ("human", "{input}"), # 사용자 입력
            MessagesPlaceholder(variable_name="agent_scratchpad"), # 에이전트의 생각/행동 기록
        ])

        # OpenAI Tools Agent 생성
        # 이 에이전트는 LLM이 도구를 사용할지, 어떤 도구를 사용할지, 어떤 입력을 줄지 결정하게 함
        agent = create_openai_tools_agent(llm, tools, prompt_template)
        print("[성공] OpenAI Tools Agent 생성 완료.")

        # Agent Executor 생성 (실제 ReAct 루프 실행)
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True, # 에이전트의 생각 과정을 보려면 True로 설정
            handle_parsing_errors=True # LLM 출력 파싱 오류 처리
        )
        print("[성공] Agent Executor 생성 완료.")

    except Exception as e:
        print(f"!! [오류] 에이전트 설정 중 오류 발생: {e}")
        traceback.print_exc()
        agent_executor = None

elif not ('llm' in locals() and llm is not None):
     print("!! [오류] LLM이 준비되지 않아 에이전트를 설정할 수 없습니다.")
elif not ('rag_tool' in locals() and rag_tool is not None):
     print("!! [오류] RAG Tool이 준비되지 않아 에이전트를 설정할 수 없습니다.")

print("--- 단계 10.2: 에이전트 설정 완료 ---")


--- 단계 10.2: ReAct 에이전트 설정 시작 ---
[정보] 에이전트가 사용할 도구: ['KoreanLaborLawQASystem']
[성공] OpenAI Tools Agent 생성 완료.
[성공] Agent Executor 생성 완료.
--- 단계 10.2: 에이전트 설정 완료 ---


# 단계 11: ReAct 에이전트 실행

In [ ]:
# === 단계 11 (수정): ReAct 에이전트 실행 ===

print("\n--- 단계 11 (수정): ReAct 에이전트 실행 시작 ---")

# 이전 단계에서 정의된 query 변수 사용 또는 새 질문 정의
# query = "근로자를 해고하려면 몇일 전에 예고를 해야 하나요?" # 예시 질문
test_query_1 = "근로자를 해고하려면 며칠 전에 예고를 해야 하나요?"
test_query_2 = "오늘 날씨 어때?" # RAG 도구를 사용하지 않아야 하는 질문 예시

queries_to_test = [test_query_1, test_query_2]
agent_results = {}

if agent_executor:
    for query in queries_to_test:
        print(f"\n\n=========================================")
        print(f"질문 입력: {query}")
        print(f"=========================================")
        agent_results[query] = None # 결과 초기화
        try:
            print("에이전트 실행 중...")
            start_agent_time = time.time()

            # 에이전트 실행!
            response = agent_executor.invoke({"input": query})

            end_agent_time = time.time()
            print(f"\n에이전트 실행 완료 ({end_agent_time - start_agent_time:.2f}초)")

            # 최종 답변 출력
            final_answer = response.get("output", "N/A")
            agent_results[query] = final_answer
            print("\n[에이전트 최종 답변]:")
            print(final_answer)

            # (참고) verbose=True 설정 시, 실행 과정에서 Thought, Action, Observation이 출력됨

        except Exception as e:
            print(f"!! [오류] 에이전트 실행 중 오류 발생: {e}")
            traceback.print_exc()

else:
    print("!! Agent Executor가 준비되지 않았습니다. 실행 불가.")

print("\n--- 단계 11 (수정): ReAct 에이전트 실행 완료 ---")


--- 단계 11 (수정): ReAct 에이전트 실행 시작 ---


질문 입력: 근로자를 해고하려면 며칠 전에 예고를 해야 하나요?
에이전트 실행 중...


> Entering new AgentExecutor chain...
근로자를 해고하려면 근로기준법에 따라 일정 기간 전에 예고를 해야 합니다. 일반적으로 해고 예고 기간은 최소 30일입니다. 즉, 해고를 통보하기 최소 30일 전에 근로자에게 예고해야 하며, 그렇지 않을 경우 30일분의 평균임금을 해고 예고수당으로 지급해야 합니다.

혹시 구체적인 상황이나 해고 사유에 따라 달라질 수 있으니, 더 자세한 법률 상담이 필요하시면 말씀해 주세요.

> Finished chain.

에이전트 실행 완료 (1.34초)

[에이전트 최종 답변]:
근로자를 해고하려면 근로기준법에 따라 일정 기간 전에 예고를 해야 합니다. 일반적으로 해고 예고 기간은 최소 30일입니다. 즉, 해고를 통보하기 최소 30일 전에 근로자에게 예고해야 하며, 그렇지 않을 경우 30일분의 평균임금을 해고 예고수당으로 지급해야 합니다.

혹시 구체적인 상황이나 해고 사유에 따라 달라질 수 있으니, 더 자세한 법률 상담이 필요하시면 말씀해 주세요.


질문 입력: 오늘 날씨 어때?
에이전트 실행 중...


> Entering new AgentExecutor chain...
죄송하지만, 저는 현재 실시간 날씨 정보를 제공할 수 없습니다. 최신 날씨 정보를 확인하시려면 날씨 앱이나 웹사이트를 이용하시거나, 스마트폰의 날씨 서비스를 이용해 주세요. 다른 도움이 필요하시면 말씀해 주세요!

> Finished chain.

에이전트 실행 완료 (1.00초)

[에이전트 최종 답변]:
죄송하지만, 저는 현재 실시간 날씨 정보를 제공할 수 없습니다. 최신 날씨 정보를 확인하시려면 날씨 앱이나 웹사이트를 이용하시거나, 스마트폰의 날씨 서비스를 이용해 주세요. 다른 도움이 필요하시면 말씀해 주세요!

--- 단계 11 (수정): ReAct 에이전트 실행 

RAG 기반 에이전트 평가에서는 항상 "Agent가 RAG 도구를 실제로 사용했는가?" 를 먼저 확인

✅ 1. intermediate_steps에 context가 있는지 먼저 확인

✅ 2. 평가 가능한 질문만 골라서 평가

[] 나오는 경우는 rag를 사용하지 않아 context가 없다는 의미

In [ ]:
from pprint import pprint
pprint(response.get("intermediate_steps", []), width=150)


[]


# ✅ 회고

ReAct(Reason + Act) 프레임워크 기반의 에이전트(Agent)를 구현해 보았습니다.
LLM이 판단하여 컨텍스트 사용여부, 답변 생성을 하는 과정에서 LLM의 한계가 적용될 수 밖에 없습니다.

첫 번째 테스트 질문인 "근로자를 해고하려면 며칠 전에 예고를 해야 하나요?"에 생성된 응답은 다음과 같습니다.

(답변)
근로자를 해고하려면 근로기준법에 따라 일정 기간 전에 예고를 해야 합니다. 일반적으로 해고 예고 기간은 최소 30일입니다. 즉, 해고를 통보하기 최소 30일 전에 근로자에게 예고해야 하며, 그렇지 않을 경우 30일분의 평균임금을 해고 예고수당으로 지급해야 합니다.

혹시 구체적인 상황이나 해고 사유에 따라 달라질 수 있으니, 더 자세한 법률 상담이 필요하시면 말씀해 주세요.

생성된 답변에서 '평균임금이 아니라 통상임금'으로 오류가 발생했습니다.

현재의 RAG agent는 참고 문서 자체를 사용하지 않을 수도 있고, 첫 번째 테스트부터 오답이 발생하여 법률 도메인에 적합하지 않다고 판단하여 더 이상 개선하지 않았습니다. 하지만 프롬프트를 좀 더 면밀하게 수정한다면, 생성된 응답의 정확도를 향상시킬 수 있을 것으로 보입니다.
